In [1]:
from utils import build_url, load_json, save_json, call_with_retry, save_execution_state, load_execution_state
import os, shutil

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = [420, 430, 440]  # Ver https://developer.riotgames.com/docs/lol#general_queue-ids

with open(API_KEY_FILE) as f:
    api_key = f.read()

execution_cache_dir = 'current_execution'
games_ids_state = f'{execution_cache_dir}/games.pkl'
accounts_state = f'{execution_cache_dir}/accounts.pkl'

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

headers = {
  'X-Riot-Token': api_key
}

accounts = load_json('outputs/accounts.json')
print(accounts[0:5])
accounts_ids = list(map(lambda a: a['account'], accounts))
print(accounts_ids[0:10])

[{'summoner': 'ug7rHf6jjFmu5QBJFbcNyexPii7WldUpyY83a2TxpX2oj7KI', 'account': 'p1o2pgpOh-tEFBZIRVzdFn0GUxVDY2B3BgsUqhSvQp8ZV54'}, {'summoner': 'YAVNENf3eu3aW2BFrPa6iWqMXcoLBOUxdp6JSIuNDo4LbvZB', 'account': 'uhYkddVgawsd8hwD4Ab1wMZshbjVJspyW__Py3EWl3QD-Bnrv-xtuMNn'}, {'summoner': 'AlK6oorIY5k7bVgFHr__uZv4sxWUlxCYkZ7C8Y3WpndocTY', 'account': '-CImT73qZ2mwJ4dFNsE8iA1JnjD4G14lk8VRP8D4vEvJcw'}, {'summoner': 'HtT7hNmQn2kwFRf7gByz0w5R44jzvAZge-6WoH7WVFn6LTc', 'account': 'LDjQWs18pbpqn-HMkKeKJmanXeCP_yA2JSROWHfYEDePkg'}, {'summoner': 'mY0n-Zr_qUezzMo6jaFvC-BNxFr5L1QWiwpUEVYVrJjH-lCC', 'account': 'sdfs8WqYZ7SPdawXCDA07cxp76vURWRdMLYK6QNzvMfcZUiG92-hUVh3'}]
['p1o2pgpOh-tEFBZIRVzdFn0GUxVDY2B3BgsUqhSvQp8ZV54', 'uhYkddVgawsd8hwD4Ab1wMZshbjVJspyW__Py3EWl3QD-Bnrv-xtuMNn', '-CImT73qZ2mwJ4dFNsE8iA1JnjD4G14lk8VRP8D4vEvJcw', 'LDjQWs18pbpqn-HMkKeKJmanXeCP_yA2JSROWHfYEDePkg', 'sdfs8WqYZ7SPdawXCDA07cxp76vURWRdMLYK6QNzvMfcZUiG92-hUVh3', 'aWaU609yXnQKVtVUsPyL5COGEeLAPFJfLhInXpe1C6t8_jHk7nRozqRo', 'q1jHew6v0dAb

In [2]:
endpoint = '/lol/match/v4/matchlists/by-account/{}'
params = {
    'queue': QUEUE_TYPE,
}

games_ids = load_execution_state(games_ids_state, set())
accounts_with_matches = load_execution_state(accounts_state, set())
accounts_ids = set(accounts_ids) - accounts_with_matches

for account_id in accounts_ids:
    full_url = build_url(REGION, BASE_URL, endpoint, account_id)
    response = call_with_retry(full_url, headers, params)
    for match in response['matches']:
        games_ids.add(match['gameId'])
    save_execution_state(games_ids, games_ids_state)
    accounts_with_matches.add(account_id)
    save_execution_state(accounts_with_matches, accounts_state)
        
save_json(list(games_ids), 'games_ids.json')

In [3]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)